In [202]:
# %%
import os,sys,inspect
currentdir = "D:\BAMT\plan\\big_cluster"
parentdir = os.path.dirname(currentdir)
parentparentdir = os.path.dirname(parentdir)
sys.path.insert(0,parentparentdir)

# %%
import pandas as pd
import random
import numpy as np
from bayesian.save_bn import save_structure, save_params, read_structure, read_params

from copy import copy


import matplotlib.pyplot as plt
from core.core_utils import project_root


from sklearn.metrics import accuracy_score, mean_squared_error

from graph.precision_recall import child_dict, precision_recall
import json5


In [203]:
# option = ''
# bad_option = ['MI', 'k2', 'K2']
option = 'MI'
bad_option = ['BIC', 'k2', 'K2']

In [204]:
name = 'pathfinder'
values = [1500, 3000]

In [205]:
ind = 0
time_list = []
suff = '_'
if len(option) > 1:
    suff = f'_{option}_'

with open(f'{currentdir}/log_full{suff}{name}.txt', 'r') as log:
    for i, line in enumerate(log):
        if i % 3 == 2:
            time_long = float(line.split(' ')[-1].rstrip())
            time_list.append(time_long)


In [206]:
time_list

[228.975501537323, 222.2513611316681]

In [207]:
coded_dict = dict()   
suff = '_'
if len(option) > 1:
    suff = f'_{option}_'
else:
    suff = f'_BIC_'

with open(f'{currentdir}/log_coded_{name}{suff}extra.txt', 'r') as log:
    for i, line in enumerate(log):
        if i % 5 == 1:
            alpha = float(line.rstrip())
        elif i % 5 == 2:
            code_type = line.rstrip()
        else:
            if i % 5 == 3:
                samples = int(line.split(' ')[1].rstrip())
            elif i % 5 == 4:
                time_long = float(line.split(' ')[-1].rstrip())
                coded_dict[str({'name': name, 'alpha': alpha, 'code_type': code_type, 'samples': samples})] = time_long
                

In [208]:
coded_dict

{"{'name': 'pathfinder', 'alpha': 0.4, 'code_type': 'hamming', 'samples': 1500}": 5.423717975616455,
 "{'name': 'pathfinder', 'alpha': 0.2, 'code_type': 'mostly', 'samples': 3000}": 368.66749262809753,
 "{'name': 'pathfinder', 'alpha': 0.2, 'code_type': 'hamming', 'samples': 3000}": 400.57999563217163,
 "{'name': 'pathfinder', 'alpha': 0.1, 'code_type': 'hamming', 'samples': 1500}": 368.4450001716614,
 "{'name': 'pathfinder', 'alpha': 0.3, 'code_type': 'mostly', 'samples': 3000}": 349.6907682418823,
 "{'name': 'pathfinder', 'alpha': 0.1, 'code_type': 'mostly', 'samples': 1500}": 343.25765013694763,
 "{'name': 'pathfinder', 'alpha': 0.1, 'code_type': 'hamming', 'samples': 3000}": 405.0489993095398,
 "{'name': 'pathfinder', 'alpha': 0.2, 'code_type': 'mostly', 'samples': 1500}": 330.05605602264404,
 "{'name': 'pathfinder', 'alpha': 0.8, 'code_type': 'hamming', 'samples': 1500}": 1.9073486328125e-06,
 "{'name': 'pathfinder', 'alpha': 0.5, 'code_type': 'hamming', 'samples': 1500}": 58.7097

In [209]:
result = pd.DataFrame(columns=['name', 'alpha', 'code_type', 'samples', 'AP', 'AR', 'AHP', 'AHR', 'SHD', 'pred_edges_len', 'true_edges_len', 'time'])

In [210]:
for root, dirs, files in os.walk(os.path.join(parentparentdir, "models", "structure_bn")):
    for file in files:
        if (name in file) and (option in file) and (('uncoded' in file) or ('_extra' in file)) and all([opt not in file for opt in bad_option]):
            att = file.split('_')
            bn = read_structure(file.replace('.txt', ''))
            if len(bn.E) > 0:
                corr_edges = [e for e in bn.E if not any([')_in' in node or ')_out' in node for node in e])]
                true_edges = read_structure(f'{name}_true').E
                true_edges_len = len(read_structure(f'{name}_true').E)
                net_stat = precision_recall(corr_edges, true_edges)
                if att[0] == 'coded':
                    cur_time = -1.0
                    alpha = float('0.' + att[2])
                    code_type = att[3]
                    samples = int(att[4].replace('.txt', ''))
                    id = {'name': name, 'alpha': alpha, 'code_type': code_type, 'samples': samples}
                    
                    for key, val in coded_dict.items():
                        dict_key = json5.loads(key)
                        if id == dict_key:
                            cur_time = val
                    if cur_time >= 0:
                        result = result.append({**id, **net_stat, 'pred_edges_len': len(corr_edges), 'true_edges_len': true_edges_len, 'time': cur_time}, ignore_index=True)
                if att[0] == 'uncoded':
                    for i, value in enumerate(values):
                        result = result.append({'name': name, 'alpha': 0.0, 'code_type': 'none', 'samples': value, **net_stat, 'pred_edges_len': len(corr_edges), 'true_edges_len': true_edges_len, 'time': time_list[i]}, ignore_index=True)
            

In [211]:
result.head()
# result

,name,alpha,code_type,samples,AP,AR,AHP,AHR,SHD,pred_edges_len,true_edges_len,time
0,pathfinder,0.1,hamming,1500,0.02,0.01,0.0,0.0,251,57,195,368.445000
1,pathfinder,0.1,hamming,3000,0.02,0.01,0.0,0.0,251,57,195,405.048999
2,pathfinder,0.1,mostly,1500,0.02,0.01,0.0,0.0,245,51,195,343.257650
3,pathfinder,0.1,mostly,3000,0.02,0.01,0.0,0.0,245,51,195,381.943152
4,pathfinder,0.2,hamming,1500,0.02,0.01,0.0,0.0,251,57,195,361.073870


In [212]:
result.to_csv(f'{name}_{option}_extra_stat.csv', index=False)

In [213]:
# true_net = "[[\"alcoholism\", \"THepatitis\"], [\"alcoholism\", \"Steatosis\"], [\"vh_amn\", \"ChHepatitis\"], [\"vh_amn\", \"hbsag\"], [\"vh_amn\", \"hbsag_anti\"], [\"vh_amn\", \"hbc_anti\"], [\"vh_amn\", \"hcv_anti\"], [\"vh_amn\", \"hbeag\"], [\"hepatotoxic\", \"THepatitis\"], [\"hepatotoxic\", \"RHepatitis\"], [\"THepatitis\", \"fatigue\"], [\"THepatitis\", \"phosphatase\"], [\"THepatitis\", \"inr\"], [\"THepatitis\", \"hepatomegaly\"], [\"THepatitis\", \"alt\"], [\"THepatitis\", \"ast\"], [\"THepatitis\", \"ggtp\"], [\"THepatitis\", \"anorexia\"], [\"THepatitis\", \"nausea\"], [\"THepatitis\", \"spleen\"], [\"hospital\", \"injections\"], [\"hospital\", \"transfusion\"], [\"surgery\", \"injections\"], [\"surgery\", \"transfusion\"], [\"gallstones\", \"choledocholithotomy\"], [\"gallstones\", \"bilirubin\"], [\"gallstones\", \"upper_pain\"], [\"gallstones\", \"fat\"], [\"gallstones\", \"pressure_ruq\"], [\"gallstones\", \"flatulence\"], [\"gallstones\", \"amylase\"], [\"choledocholithotomy\", \"injections\"], [\"choledocholithotomy\", \"transfusion\"], [\"injections\", \"ChHepatitis\"], [\"transfusion\", \"ChHepatitis\"], [\"ChHepatitis\", \"fibrosis\"], [\"ChHepatitis\", \"fatigue\"], [\"ChHepatitis\", \"bilirubin\"], [\"ChHepatitis\", \"pressure_ruq\"], [\"ChHepatitis\", \"phosphatase\"], [\"ChHepatitis\", \"inr\"], [\"ChHepatitis\", \"ESR\"], [\"ChHepatitis\", \"alt\"], [\"ChHepatitis\", \"ast\"], [\"ChHepatitis\", \"ggtp\"], [\"ChHepatitis\", \"cholesterol\"], [\"ChHepatitis\", \"hbsag\"], [\"ChHepatitis\", \"hbsag_anti\"], [\"ChHepatitis\", \"hbc_anti\"], [\"ChHepatitis\", \"hcv_anti\"], [\"ChHepatitis\", \"hbeag\"], [\"sex\", \"PBC\"], [\"sex\", \"Hyperbilirubinemia\"], [\"age\", \"PBC\"], [\"age\", \"Hyperbilirubinemia\"], [\"PBC\", \"bilirubin\"], [\"PBC\", \"pressure_ruq\"], [\"PBC\", \"ama\"], [\"PBC\", \"le_cells\"], [\"PBC\", \"joints\"], [\"PBC\", \"pain\"], [\"PBC\", \"platelet\"], [\"PBC\", \"encephalopathy\"], [\"PBC\", \"ESR\"], [\"PBC\", \"ggtp\"], [\"PBC\", \"cholesterol\"], [\"PBC\", \"carcinoma\"], [\"fibrosis\", \"Cirrhosis\"], [\"diabetes\", \"obesity\"], [\"obesity\", \"Steatosis\"], [\"Steatosis\", \"Cirrhosis\"], [\"Steatosis\", \"triglycerides\"], [\"Steatosis\", \"pain_ruq\"], [\"Steatosis\", \"hepatomegaly\"], [\"Steatosis\", \"ESR\"], [\"Steatosis\", \"alt\"], [\"Steatosis\", \"ast\"], [\"Steatosis\", \"ggtp\"], [\"Steatosis\", \"cholesterol\"], [\"Cirrhosis\", \"bilirubin\"], [\"Cirrhosis\", \"phosphatase\"], [\"Cirrhosis\", \"proteins\"], [\"Cirrhosis\", \"edema\"], [\"Cirrhosis\", \"platelet\"], [\"Cirrhosis\", \"inr\"], [\"Cirrhosis\", \"alcohol\"], [\"Cirrhosis\", \"encephalopathy\"], [\"Cirrhosis\", \"alt\"], [\"Cirrhosis\", \"ast\"], [\"Cirrhosis\", \"spleen\"], [\"Cirrhosis\", \"spiders\"], [\"Cirrhosis\", \"albumin\"], [\"Cirrhosis\", \"edge\"], [\"Cirrhosis\", \"irregular_liver\"], [\"Cirrhosis\", \"palms\"], [\"Cirrhosis\", \"carcinoma\"], [\"Hyperbilirubinemia\", \"bilirubin\"], [\"Hyperbilirubinemia\", \"pain_ruq\"], [\"Hyperbilirubinemia\", \"inr\"], [\"Hyperbilirubinemia\", \"hepatomegaly\"], [\"Hyperbilirubinemia\", \"ESR\"], [\"Hyperbilirubinemia\", \"ggtp\"], [\"RHepatitis\", \"fatigue\"], [\"RHepatitis\", \"phosphatase\"], [\"RHepatitis\", \"hepatomegaly\"], [\"RHepatitis\", \"alt\"], [\"RHepatitis\", \"ast\"], [\"RHepatitis\", \"ggtp\"], [\"RHepatitis\", \"anorexia\"], [\"RHepatitis\", \"nausea\"], [\"RHepatitis\", \"spleen\"], [\"bilirubin\", \"itching\"], [\"bilirubin\", \"skin\"], [\"bilirubin\", \"jaundice\"], [\"joints\", \"pain\"], [\"proteins\", \"ascites\"], [\"platelet\", \"bleeding\"], [\"inr\", \"bleeding\"], [\"encephalopathy\", \"urea\"], [\"encephalopathy\", \"density\"], [\"encephalopathy\", \"consciousness\"], [\"hepatomegaly\", \"hepatalgia\"], [\"hbsag\", \"hbsag_anti\"]]"
# true_net = json5.loads(true_net)
# print(true_net)

In [214]:
# nodes = []
# for e in true_net:
#     nodes.extend(e)
# nodes = list(set(nodes))

In [215]:
# save_structure({'V': nodes, 'E': true_net}, 'hepar2_true')